<a href="https://colab.research.google.com/github/dleqhuy/Sentiment_Analysis/blob/main/02.pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mô tả và đánh giá dữ liệu & Tiền xử lí dữ liệu

In [ ]:
!apt-get update
!apt install -qq enchant

!git clone https://github.com/dleqhuy/Sentiment_Analysis
!pip install -r /content/Sentiment_Analysis/requirements.txt

In [2]:
import sys
sys.path.append('/content/Sentiment_Analysis')
%load_ext autoreload
%autoreload 2


In [3]:
import modules.utils as Utils
import modules.processor as Processor
import numpy as np
import pandas as pd
import enchant
import random

from sklearn.utils import shuffle

In [4]:
reviews = pd.read_csv('/content/drive/MyDrive/shopee/product_reviews.csv') 

In [5]:
reviews.head()

,comment,rating
0,Mùi hương:không\nDành cho da:mọi loại da\n\nPh...,5
1,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,5
2,Dành cho da:mọi loại da\nCông dụng:rửa mặt\nMù...,5
3,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,2
4,Công dụng:Sạch sẽ\nMùi hương:dễ chịu\nDành cho...,5


In [6]:
print("Tập dữ liệu có {} bình luận.".format(reviews.shape[0]))

Tập dữ liệu có 214329 bình luận.


* Đếm tần số xuất hiện của từng rating.

In [7]:
reviews['rating'].value_counts()

5    204727
4      5256
3      2000
1      1537
2       809
Name: rating, dtype: int64

> **Nhận xét**:
> * Nhìn chung tuy ta crawl được hơn 200,000 quan sát nhưng có sự chênh lệch lớn giữa các rating.
> * Nhìn qua ta thấy đa phần là các rating được đánh giá 5 sao, với những đánh giá này ta có thể đưa chúng vào nhóm positive, bởi vì những comment này chứa những thông tin positive mặt khác chúng cũng có thể chứa những thông tin negative nhưng vì được đánh giá 5 sao nên thông tin negative trong nhóm này có thể ít. 
> * Tương tự với những đánh giá 4 sao cũng vậy, có thể sản phẩm có một ít vấn đề nên đánh giá 4 sao, nhưng tổng thể thì thông tin positive > negative

> * Với những đánh giá dưới 4 sao thì sản phẩm có vấn đề nên đánh giá thấp sao, và có thể những comment chứa thông tin negative nhiều hơn positive

* Tiến hành label cho `reviews` với các giá `rating` $< 4$ sẽ thuộc nhóm negative còn lại là nhóm positive.

In [8]:
reviews['label'] = reviews['rating'].apply(lambda rt: 1 if rt >= 4 else 0)
print(reviews['label'].value_counts())
reviews.head()

1    209983
0      4346
Name: label, dtype: int64


,comment,rating,label
0,Mùi hương:không\nDành cho da:mọi loại da\n\nPh...,5,1
1,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,5,1
2,Dành cho da:mọi loại da\nCông dụng:rửa mặt\nMù...,5,1
3,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,2,0
4,Công dụng:Sạch sẽ\nMùi hương:dễ chịu\nDành cho...,5,1


In [9]:
reviews = reviews.drop(columns=['rating'])
reviews.head()

,comment,label
0,Mùi hương:không\nDành cho da:mọi loại da\n\nPh...,1
1,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,1
2,Dành cho da:mọi loại da\nCông dụng:rửa mặt\nMù...,1
3,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,0
4,Công dụng:Sạch sẽ\nMùi hương:dễ chịu\nDành cho...,1


* Bây giờ, ta sẽ 
  * `lower()` cho text.

In [10]:
reviews['normalize_comment'] = reviews['comment'].apply(lambda cmt: Processor.normalizeComment(cmt))
reviews.head()

,comment,label,normalize_comment
0,Mùi hương:không\nDành cho da:mọi loại da\n\nPh...,1,mùi hương:không\ndành cho da:mọi loại d...
1,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,1,công dụng:rửa mặt\nmùi hương:không m...
2,Dành cho da:mọi loại da\nCông dụng:rửa mặt\nMù...,1,dành cho da:mọi loại da\ncông dụng:rửa ...
3,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,0,công dụng:rửa mặt\nmùi hương:không m...
4,Công dụng:Sạch sẽ\nMùi hương:dễ chịu\nDành cho...,1,công dụng:sạch sẽ\nmùi hương:dễ chịu...


<hr>

* Các comment của các sản phẩm đôi khi sẽ chứa các URL do người bán hàng chèn vào để giúp khách hàng có thể click vào để xem các mặt hàng khác, chúng là các noise sample mà ta cần phải loại bỏ khỏi dataset của chúng ta.<br>

In [11]:
reviews['contain_url'] = reviews['normalize_comment'].apply(lambda cmt: Processor.containsURL(cmt))
print(reviews['contain_url'].value_counts())
reviews.head()

0    213745
1       584
Name: contain_url, dtype: int64


,comment,label,normalize_comment,contain_url
0,Mùi hương:không\nDành cho da:mọi loại da\n\nPh...,1,mùi hương:không\ndành cho da:mọi loại d...,0
1,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,1,công dụng:rửa mặt\nmùi hương:không m...,0
2,Dành cho da:mọi loại da\nCông dụng:rửa mặt\nMù...,1,dành cho da:mọi loại da\ncông dụng:rửa ...,0
3,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,0,công dụng:rửa mặt\nmùi hương:không m...,0
4,Công dụng:Sạch sẽ\nMùi hương:dễ chịu\nDành cho...,1,công dụng:sạch sẽ\nmùi hương:dễ chịu...,0


* Bây giờ chúng ta chỉ sẽ lấy các comment mà không chứa URL

In [12]:
reviews = reviews[reviews['contain_url'] == 0]
reviews = reviews.drop(columns=['contain_url']).reset_index(drop=True) # xóa cột `contain_url`
print(reviews['label'].value_counts())
reviews.head()

1    209405
0      4340
Name: label, dtype: int64


,comment,label,normalize_comment
0,Mùi hương:không\nDành cho da:mọi loại da\n\nPh...,1,mùi hương:không\ndành cho da:mọi loại d...
1,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,1,công dụng:rửa mặt\nmùi hương:không m...
2,Dành cho da:mọi loại da\nCông dụng:rửa mặt\nMù...,1,dành cho da:mọi loại da\ncông dụng:rửa ...
3,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,0,công dụng:rửa mặt\nmùi hương:không m...
4,Công dụng:Sạch sẽ\nMùi hương:dễ chịu\nDành cho...,1,công dụng:sạch sẽ\nmùi hương:dễ chịu...


> **Nhận xét**
> * Đa phần là các bình luận thuộc nhóm positive sẽ chứa các URL

In [13]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.decodeEmoji(cmt))
reviews.head()

,comment,label,normalize_comment
0,Mùi hương:không\nDành cho da:mọi loại da\n\nPh...,1,mùi hương:không\ndành cho da:mọi loại d...
1,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,1,công dụng:rửa mặt\nmùi hương:không m...
2,Dành cho da:mọi loại da\nCông dụng:rửa mặt\nMù...,1,dành cho da:mọi loại da\ncông dụng:rửa ...
3,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,0,công dụng:rửa mặt\nmùi hương:không m...
4,Công dụng:Sạch sẽ\nMùi hương:dễ chịu\nDành cho...,1,công dụng:sạch sẽ\nmùi hương:dễ chịu...


<hr>

* Tiếp theo, ta sẽ loại bỏ dấu câu, kí tự đặc biệt

In [14]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeSpecialLetters(cmt))
reviews.head()

,comment,label,normalize_comment
0,Mùi hương:không\nDành cho da:mọi loại da\n\nPh...,1,mùi hương không dành cho da mọi loại da...
1,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,1,công dụng rửa mặt mùi hương không mu...
2,Dành cho da:mọi loại da\nCông dụng:rửa mặt\nMù...,1,dành cho da mọi loại da công dụng rửa m...
3,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,0,công dụng rửa mặt mùi hương không mu...
4,Công dụng:Sạch sẽ\nMùi hương:dễ chịu\nDành cho...,1,công dụng sạch sẽ mùi hương dễ chịu ...


<hr>

* Tiếp theo, ta cần chuẩn lại các từ bị dupplicate như: _chờiiiiiii ơiiiiiii, xinhhhhhhh quá, đẹp xỉuuuuuuuuuuuu_ thành _chời ơi, xinh quá, đẹp xỉu_.
* Tuy nhiên có một vấn đề xảy ra, giả sử trong comment có các từ tiếng anh như "_feedback_", thì nó sẽ thành "_fedback_", nên ta sẽ thực hiện bước này ở phần sau:

```python
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeDuplicateLetters(cmt))

```

<hr>

* Tiếp theo, chúng ta sẽ chuẩn lại một vài từ viết tắt cơ bản.
* File `modules/dependencies/abbreviate.txt` chứa các từ viết tắt cơ bản mà giới trẻ hay dùng comment, ta có thể bổ sung theo thời gian.

In [15]:
# xây dựng dictionary cho các từ viết tắt
abbreviate = Utils.buildDictionaryFromFile("/content/Sentiment_Analysis/modules/dependencies/abbreviate.txt")

# test
abbreviate['okela']

'ok'

In [16]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.replaceWithDictionary(cmt, abbreviate))

reviews.head()

,comment,label,normalize_comment
0,Mùi hương:không\nDành cho da:mọi loại da\n\nPh...,1,mùi hương không dành cho da mọi loại da...
1,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,1,công dụng rửa mặt mùi hương không mu...
2,Dành cho da:mọi loại da\nCông dụng:rửa mặt\nMù...,1,dành cho da mọi loại da công dụng rửa m...
3,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,0,công dụng rửa mặt mùi hương không mu...
4,Công dụng:Sạch sẽ\nMùi hương:dễ chịu\nDành cho...,1,công dụng sạch sẽ mùi hương dễ chịu ...


<hr>

* Bây giờ ta sẽ tiến hành xóa các sample mà khả năng cao ko là tiếng việt, vì sao ta làm bước này, đơn giản thôi đây là shopee việt nam, và các comment cố ý bằng tiếng anh, tiếng hàn, tiếng trung sẽ là các noise sample khiến model ta bị giảm hiệu năng.
* Nhưng làm sao ta có thể thực hiện điều này, cách đơn giản nhất là ta có thể sử dụng các package như `textblob`, `googletrans`,... các package này chứa các function giúp ta detect language cho text, tuy nhiên hạn chế là chúng chỉ cho tối đa khoảng 200 request một ngày thôi, và số mẫu của chúng ta hiện tại là quá lớn. Ở đây ta có file `modules/dependencies/vocabulary.txt` chứa hơn 17000 từ đơn phổ biến của tiếng việt.
* Vậy cách đơn giản hơn là ta có thể xây dựng một dictionary chứa các từ đơn của tiếng việt, với mỗi comment, nếu số lượng từ ko tìm thấy trong dictionary này lớn hơn số từ được tìm thấy trong dictionary thì khả năng cao đây là một comment làm màu.
* Tuy nhiên, vẫn có một vài từ tiếng anh mà ta cần giữ lại như shipper, ta sẽ sử dụng package `enchant` để check một từ có phải là từ tiếng anh hay không.
  ```shell
  pip3 install pyenchant
  ```
* Ở các bước phía trên, ta đã đề cập đến việc xóa các từ bị dupplicate kí tự, ta sẽ thực hiện nó ở trong bước này.

In [17]:
# hơn 17 ngàn từ đơn trong tiêng việt
vocabularies = Utils.buildDictionaryFromFile('/content/Sentiment_Analysis/modules/dependencies/vocabulary.txt', True)
english_voca = enchant.Dict('en_US') # english if a word is english

In [18]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeNoiseWord(cmt, vocabularies, english_voca))

reviews.head()

,comment,label,normalize_comment
0,Mùi hương:không\nDành cho da:mọi loại da\n\nPh...,1,mùi hương không dành cho da mọi loại da...
1,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,1,công dụng rửa mặt mùi hương không mu...
2,Dành cho da:mọi loại da\nCông dụng:rửa mặt\nMù...,1,dành cho da mọi loại da công dụng rửa m...
3,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,0,công dụng rửa mặt mùi hương không mu...
4,Công dụng:Sạch sẽ\nMùi hương:dễ chịu\nDành cho...,1,công dụng sạch sẽ mùi hương dễ chịu ...


<hr>

* Tiếp theo ta sẽ remove stopword, chúng ta sẽ sử dụng stopword trong file `modules/dependencies/stopwords.txt`. Ta không nên sử dụng các stopword được build sẵn trên mạng nhất là cho tiếng việt, vì chưa chắc các từ này đã hợp với dữ liệu hiện tại của chúng ta.
* Ví dụ nhiều stopword set loại bỏ từ "**nhưng**", tuy nhiên từ này khả năng cao là quan trọng, giả sử ta có câu này:
  * _shop giao hàng chậm **nhưng** giao đúng hàng, ủng hộ shop_, thì nhờ từ **nhưng** này mà model ta có khả năng phân biệt được nó là positive hay negative.
* Ngoài ra với một file txt như vậy, ta có thể bổ sung stopword sau này.

In [19]:
stopwords = Utils.buildListFromFile("/content/Sentiment_Analysis/modules/dependencies/stopwords.txt")

In [20]:
reviews['normalize_comment'] = reviews['normalize_comment'].apply(lambda cmt: Processor.removeStopwords(cmt, stopwords))

reviews.head()

,comment,label,normalize_comment
0,Mùi hương:không\nDành cho da:mọi loại da\n\nPh...,1,mùi hương không dành cho da mọi loại da...
1,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,1,công dụng rửa mặt mùi hương không mu...
2,Dành cho da:mọi loại da\nCông dụng:rửa mặt\nMù...,1,dành cho da mọi loại da công dụng rửa m...
3,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,0,công dụng rửa mặt mùi hương không mu...
4,Công dụng:Sạch sẽ\nMùi hương:dễ chịu\nDành cho...,1,công dụng sạch sẽ mùi hương dễ chịu ...


<hr>

* Tiếp theo, ta loại bỏ các empty và duplicate `normalize_comment`.

In [21]:
reviews = Processor.removeEmptyOrDuplicateComment(reviews)
print(reviews['label'].value_counts())

reviews.head()

1    195076
0      4205
Name: label, dtype: int64


,comment,label,normalize_comment
0,Mùi hương:không\nDành cho da:mọi loại da\n\nPh...,1,mùi hương không dành cho da mọi loại da...
1,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,1,công dụng rửa mặt mùi hương không mu...
2,Dành cho da:mọi loại da\nCông dụng:rửa mặt\nMù...,1,dành cho da mọi loại da công dụng rửa m...
3,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,0,công dụng rửa mặt mùi hương không mu...
4,Công dụng:Sạch sẽ\nMùi hương:dễ chịu\nDành cho...,1,công dụng sạch sẽ mùi hương dễ chịu ...


<hr>

* Train test split, ta thấy rằng giữa hai nhóm positive và negative có chênh lệnh lớn, nên tập train data của ta sẽ bằng $0.8 * \min(\mathrm{size}(positive), \mathrm{size}(negative)) * 2$.

In [22]:
half_min_size = min(reviews['label'].value_counts())

half_min_size

4205

In [23]:
reviews_positive = reviews[reviews['label'] == 1]
reviews_negative = reviews[reviews['label'] == 0]

reviews_positive = shuffle(reviews_positive)
reviews_positive = reviews_positive.reset_index(drop=True)

In [24]:
positive_index = random.sample(range(0, reviews_positive.shape[0]), half_min_size)

positive_index[:10]

[99678, 181924, 71861, 18891, 38175, 134454, 114080, 64726, 74719, 162077]

In [25]:
reviews_positive2 = reviews_positive.iloc[positive_index,:]

reviews_positive2.head()

,comment,label,normalize_comment
99678,Giao hàng rất nhanh nha. Lúc đầu thấy cái hộp ...,1,giao hàng rất nhanh lúc đầu thấy cái ...
181924,Hiệu quả dưỡng ẩm:cao\nKhả năng thẩm thấu:cai\...,1,hiệu quả dưỡng ẩm cao khả năng thẩ...
71861,Dùng lọ thứ 3 rồi cũng tốt. Giá cả rẻ hơn các ...,1,dùng lọ thứ cũng tốt giá cả rẻ hơn ...
18891,Vì chưa dùng kem lần nào nên đang thắc mắc k b...,1,vì chưa dùng kem lần nào nên đang thắ...
38175,Công dụng:NÂng tone dưỡng sáng\nHiệu quả dưỡng...,1,công dụng nâng tone dưỡng sáng hiệu q...


* Đây là tập data mà hai nhóm positive và negative cân bằng nhau

In [26]:
normalize_reviews = pd.concat([reviews_negative, reviews_positive2], axis=0)
normalize_reviews = normalize_reviews.reset_index(drop=True)

print(normalize_reviews['label'].value_counts())

normalize_reviews.head()

0    4205
1    4205
Name: label, dtype: int64


,comment,label,normalize_comment
0,Công dụng:rửa mặt\nMùi hương:không mùi\nDành c...,0,công dụng rửa mặt mùi hương không mu...
1,Dành cho da:mọi loại da\nCông dụng:rỬa mặt\nMù...,0,dành cho da mọi loại da công dụng rửa m...
2,Công dụng:Rửa mặt\nMùi hương:không mùi\nDành c...,0,công dụng rửa mặt mùi hương không mu...
3,Công dụng:lÀm sạch\nMùi hương:bình thường\nDàn...,0,công dụng làm sạch mùi hương bình thư...
4,Mk săn sale srm đc kèm nc tẩy trang 0đ mà nhận...,0,săn sale được kèm nước tẩy trang mà...


* Ghi ra file

In [27]:
normalize_reviews.to_csv("/content/drive/MyDrive/shopee/normalize_reviews.csv", index=False)

* Bây giờ ta sẽ ghi phần bù còn lại của `review_positive` vào file, ta có thể dùng nó cho việc evaluate model sau này.

In [28]:
reviews_positive3 = reviews_positive[~reviews_positive.index.isin(positive_index)]
reviews_positive3 = reviews_positive3.reset_index(drop=True)

reviews_positive3.head()

,comment,label,normalize_comment
0,Công dụng:lm sạch mụn đầu đen\nMùi hương:chưa ...,1,công dụng lắm sạch mụn đầu đen mùi hu...
1,Trong mùa dịch thế này mà vẫn giao hàng nhanh ...,1,trong mùa dịch mà vẫn giao hàng nhanh ch...
2,"Hàng về đúng mẫu , nhưng giao hàng rất lâu nhé...",1,hàng về đúng mẫu nhưng giao hàng rất...
3,Rửa mặt rất êm. Không gây khô da. Shop giao nh...,1,rửa mặt rất êm không gây khô da giao...
4,Mua nguyên bộ 7 sản phẩm của hãng. Kem chống n...,1,mua nguyên bộ sản phẩm của hãng kem ch...


In [29]:
reviews_positive3.to_csv("/content/drive/MyDrive/shopee/complement_positive_reviews.csv", index=False)